In [ ]:
# install the latest features of the earth engine api
!pip install --upgrade earthengine-api

In [ ]:
#Import Earth Engine API and the geemap package for geospatial data visulization
import google
from google.colab import auth
from google.auth import compute_engine

import ee
import geemap

In [ ]:
# Authenticate with user credentials
auth.authenticate_user()

In [ ]:
# Please fill in these values.
PROJECT = "ee-glenromzyyali"  # @param {type:"string"} ***Enter your ee-userprojectname***

PROJECT = None if PROJECT == "" else PROJECT

# Quick input validations.
if PROJECT is None:
    import warnings
    warnings.warn("⚠️ A Google Cloud project was not provided. Please consider providing a Google Cloud project ID")

In [ ]:
# get the user credentials from the VM
credentials, _ = google.auth.default()

# initialize the client library
ee.Initialize(
    credentials,
    project=PROJECT
)
# if you do not have a cloud project due to org restrictions set project=None

In [ ]:
# create a Map object with geemap to visualize EE results
map = geemap.Map()

In [ ]:
# programically create a scratch cell for displaying the Map
from google.colab import _frontend
_frontend.create_scratch_cell("#@title Map\nmap", False)

In [ ]:
#Define Geemap basemap visualizer
map.add("basemap_selector")

In [ ]:
# Import the UAV image into your coding environment.
UAVImage = ee.Image("users/dipsgautam/20200309_alex_east_ms_orthomosaic_2cm_mga54");

In [ ]:
# Print information about the UAVImage
# In Earth Engine Python API, you don't directly "print" an ee.Image.
# Instead, you can get information about it and print that.
description = UAVImage.getInfo()
print(description)

In [ ]:
# Display the true color composite
map.add_ee_layer(UAVImage,{'bands':['b3','b2','b1'],'min':5000,'max':40000},"UAV Image True color")

# Always center the map to the UAVImage location
# You need to get the geometry of the image to center it.
# Centering requires coordinates, so we'll use the centroid of the image's geometry.
centroid = UAVImage.geometry().centroid()
coords = centroid.coordinates().getInfo()  # Get coordinates as a Python list [longitude, latitude]
map.centerObject(UAVImage, zoom=15)  # Adjust the zoom level (here, 15) as needed

In [ ]:
# Display the false color composite
map.add_ee_layer(UAVImage,{'bands':['b5','b3','b2'],'min':5000,'max':40000},"UAV Image False color")

# Define variable NDVI from equation
ndviImage = UAVImage.expression(
  "(NIR - RED) / (NIR + RED)",
  {
    'RED': UAVImage.select("b3"),    #  b3 is the RED band
    'NIR': UAVImage.select("b5")    # b5 is the NIR band
  }).rename('ndvi')

In [ ]:
# Add the NDVI image to the map, using the visualization parameters.
map.add_ee_layer(ndviImage, {'min': 0, 'max': 1, 'palette': ['red','yellow','green','darkgreen']}, "NDVI")

In [ ]:
# Mask out all the pixels that have ndvi value lower than 0.4
maskedNDVI = ndviImage.updateMask(ndviImage.gt(0.4))

# Add the maskedNDVI to the map, using the visualization parameters.
map.add_ee_layer(maskedNDVI, {'min': 0.4, 'max': 1, 'palette': ['yellow','green','darkgreen']}, "masked NDVI")

In [ ]:
# Define the coordinates of the four vines
r191v5 = ee.Geometry.Point([140.82946004591133, -37.286255642184436])
r201v11 = ee.Geometry.Point([140.82935912779715, -37.28602196707126])
r206v8 = ee.Geometry.Point([140.82943927982373, -37.28590726084841])
r206v23 = ee.Geometry.Point([140.82904924791364, -37.28587241960449])

In [ ]:
# Buffer the point geometry
r191v5_buffered = r191v5.buffer(1.5)
r201v11_buffered = r201v11.buffer(1.5)
r206v8_buffered = r206v8.buffer(1.5)
r206v23_buffered = r206v23.buffer(1.5)

In [ ]:
# display the buffered geometry
map.add_ee_layer(r191v5_buffered,{},"r191v5_buffered")
map.add_ee_layer(r201v11_buffered,{},"r201v11_buffered")
map.add_ee_layer(r206v8_buffered,{},"r206v8_buffered")
map.add_ee_layer(r206v23_buffered,{},"r206v23_buffered")

In [ ]:
# Extract the values from the canopies
ndviR191V5 = maskedNDVI.reduceRegion(
  reducer= ee.Reducer.mean(),  # We are sampling the mean NDVI value
  geometry= r191v5_buffered, # Sampling from Row-191 Vine 5
  scale= 0.02 # Sampling from pixel that are of 2 cm resolution
  )

In [ ]:
# Print the extracted mean NDVI value
# In Earth Engine Python API, you need to explicitly evaluate the result
# using getInfo() or get().
print("Mean NDVI", ndviR191V5.get('ndvi').getInfo())

In [ ]:
# implement the formula here
netPhot = maskedNDVI.multiply(95.101).add(-43.925).rename('netPhoto')
# add the estimated netPhotosynthesis map
map.add_ee_layer(netPhot,{'min':0,'max':16,'palette':['red', 'lightyellow', 'yellow', 'green', 'darkgreen']},"estimated - Net Photosynthesis")